In [17]:
import pandas as pd
import gzip
from collections import *
import networkx as nx
from geopy.distance import distance
from networkx import NetworkXError, NetworkXNoPath
import tqdm
import seaborn as sns

# Optional: Load in specific users here

# Load in the inferred users' locations (if we want physical distance)

In [37]:
user_to_loc_line = {}
with gzip.open('/shared/1/projects/location-inference/data/locations.2012-2019.from-merged.all.4-iter.tsv_iter_3.with-names.tsv.gz', 'rt') as f:
    for line_no, line in tqdm.tqdm_notebook(enumerate(f, 1), total=146000000):
        cols = line.split('\t')
        uid = int(cols[0])
        if uid in users:
            user_to_loc_line[uid] = line
        if line_no % 1000000 == 0:
            #print("saw %d lines; %d users" % (line_no, len(user_to_loc_line)))
            pass

# Load in the social network

In [38]:
g = nx.Graph()
with gzip.open('/shared/1/projects/location-inference/working-dir/mention-network.2012-2019.tsv.gz', 'rt') as f:
    for line_no, line in tqdm.tqdm_notebook(enumerate(f, 0), total=959428294):
        if line_no == 0:
            continue
        cols = line.split('\t')
        uid1 = int(cols[0])
        uid2 = int(cols[1])
        g.add_edge(uid1, uid2)
    
        if line_no % 10000000 == 0:
            #print('Loaded %d lines, saw %d nodes' % (line_no, g.number_of_nodes()))
            pass
print('Loaded %d lines, saw %d nodes' % (line_no, g.number_of_nodes()))

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




Loaded 959428294 lines, saw 153496862 nodes


In [39]:
g.number_of_nodes()

153496862

# Load in some tweets

In [ ]:
tweets = [] # In twitter JSON format

# Compute distance metrics

In [86]:
new_columns = defaultdict(list)

use_physical_location = False

for i, row in tqdm.tqdm_notebook(tweets):
    
    from_uid = int(row['id_str'])
    to_uid = int(row['in_reply_to_user_id_str'])
    
    # This is when people do threaded tweets
    if from_uid == to_uid:
        continue
        
    new_columns['tweet_id'].append(int(row['tweet_id']))

    if use_physical_location:
        try:
            from_loc = user_to_loc_line[from_uid].strip().split('\t')
            to_loc = user_to_loc_line[to_uid].strip().split('\t')

            in_same_country = from_loc[5] == to_loc[5]
            in_same_state = in_same_country and from_loc[4] == to_loc[4]
            loc1 = (float(from_loc[1]), float(from_loc[2]))
            loc2 = (float(to_loc[1]), float(to_loc[2]))
            dist = distance(loc1, loc2).km
            # print(loc1, loc2, dist)

        except KeyError as e: # Not located
            #print(repr(e))
            dist = float('nan')
            in_same_country = dist
            in_same_state = dist

        new_columns['distance'].append(dist)
        new_columns['in_same_country'].append(in_same_country)
        new_columns['in_same_state'].append(in_same_state)
    
    try:
        from_neighbors = set(nx.all_neighbors(g, from_uid))
        to_neighbors = set(nx.all_neighbors(g, to_uid))
        
        jaccard = len(from_neighbors & to_neighbors) / float(len(from_neighbors | to_neighbors))
        try:
            spl = nx.shortest_path_length(g, from_uid, to_uid)
        except NetworkXNoPath as np:
            spl = float('nan')
        aai = next(nx.adamic_adar_index(g, [(from_uid, to_uid)]))[2]
        # print(aai)
    except NetworkXError as e: # Not mentioned
        #print(repr(e))
        jaccard = float('nan')
        spl = jaccard
        aai = jaccard
        
    new_columns['jaccard_index'].append(jaccard)
    new_columns['shortest_path_length'].append(spl)
    new_columns['adamic_adar_index'].append(aai)    
    
    #if i > 5:
    #    break
new_columns = pd.DataFrame(new_columns)

In [ ]:
# drop anything we could't infer a position for
new_columns = new_columns.dropna()